In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from sklearn.linear_model import LinearRegression

import voteestimator
import numpy as np

## Read

In [ ]:
einde_stemperiode = (pd.read_excel('Data\EindeStemperiode.xlsx', engine='openpyxl')
                     .dropna(subset=['EindeStemperiode'])
                     .sort_values('EindeStemperiode')
                     .drop(columns=['Bron'])
                    )

In [ ]:
notering = pd.read_parquet(os.path.join('Data', 'notering.parquet'))
song = pd.read_parquet(os.path.join('Data', 'song.parquet'))
songartist = pd.read_parquet(os.path.join('Data', 'songartist.parquet'))
artist = pd.read_parquet(os.path.join('Data', 'artist.parquet'))

In [ ]:
def find_next_top2000_after_death(df, top2000_stemperiodes, column='Overleden'):
    not_passed_away_during_top_2000 = df[~df['IsOverleden']].copy()
    passed_away_during_top2000 = df[df['IsOverleden']].sort_values(column).copy().reset_index()
    
    passed_away_during_top2000 = pd.merge_asof(passed_away_during_top2000, einde_stemperiode,
                                               left_on=column, right_on='EindeStemperiode', direction='forward')
    df = pd.concat([not_passed_away_during_top_2000, passed_away_during_top2000.set_index('ArtistID')], sort=False)
    return df

def check_passed_away_during_top2000(df, top2000_stemperiodes, column='Overleden'):
    first_stemperiode = top2000_stemperiodes['EindeStemperiode'].min()
    relevant_date_of_death = first_stemperiode - pd.Timedelta('365 days')
    df['IsOverleden'] = df[column].ge(relevant_date_of_death)
    return df

def artist_features(df):
    columns_possible_country_of_birth = ['Oorsprong',
                                     'Land',
                                     'Land van oorsprong',
                                     'Geboorteland',
                                     'Geboorteplaats',
                                     'Nationaliteit'
                                    ]
    df = (df.assign(AgePassing = lambda df: df['Overleden'].sub(df['Geboren']).dt.days.div(365.25),
                   IsDutch = lambda df: df[columns_possible_country_of_birth].apply(lambda c: c.str.lower().str.contains('nederland')).any('columns'),
                  )
                                                
            .pipe(check_passed_away_during_top2000, einde_stemperiode)
            .pipe(find_next_top2000_after_death, einde_stemperiode)

         )

    return df

artist = artist.pipe(artist_features)

## Estimate votes per ranking

In [ ]:
m = voteestimator.MeindertsmaVotesEstimator()

In [ ]:
notering = notering.assign(PctVotes = lambda df: df['Rank'].apply(m.percentage_of_votes))

## Create analysis set

In [ ]:
def select_songs_by_one_artist(df):
    nr_artists = df['SongID'].map(songartist.groupby('SongID').size())
    df = df.loc[nr_artists.eq(1)]
    return df

In [ ]:

def normalize_by_years_before_death(df, years_to_normalize=2):
    votes_before_death = (df.loc[df['YearsSinceOverlijden'].isin(range(-years_to_normalize, 0)), ['PctVotes', 'SongID', 'YearsSinceOverlijden']]
                            .set_index(['SongID', 'YearsSinceOverlijden'])
                            .unstack(fill_value=0.5*m.percentage_of_votes(2000)).stack() # TODO: Should only use this fill value if the song already existed
                            .mean(level='SongID')
                            .rename(columns={'PctVotes': 'PctVotesBeforeDeath'})
                            .reset_index()
                        )
    df = df.merge(votes_before_death)

    return df

In [ ]:
analysis = (notering.merge(song, left_on='SongID', right_index=True)
                    .merge(songartist.reset_index())
                    .merge(artist, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
                    .pipe(select_songs_by_one_artist)
                    .assign(YearsSinceOverlijden = lambda df: df['Year'].sub(df['JaarTop2000']))
                    .pipe(normalize_by_years_before_death)
            )

analysis_overlijden = (analysis.query('YearsSinceOverlijden == 0')
                               .assign(NrsBeforeDeath = lambda df: df.groupby('ArtistID')['ArtistID'].transform('count'),
                                       PopularityWithinArtist = lambda df: df.groupby('ArtistID')['PctVotesBeforeDeath'].apply(lambda v: v.div(v.max())),
                                       RecencyWithinArtist = lambda df: df.groupby('ArtistID')['YearMade'].apply(lambda v: v.sub(v.min()).div(v.max() - v.min())),
                                       Recency = lambda df: df['YearMade'].sub(df['JaarTop2000'])
                                       Boost = lambda df: df['PctVotes'].div(df['PctVotesBeforeDeath']),
                                       )
                                .copy()
                      )
artist_overlijden = (analysis_overlijden.groupby('ArtistID').agg({'PctVotes': 'sum',
                                                                  'PctVotesBeforeDeath': 'sum',
                                                                  'YearMade': 'last'}
                                                                  )
                                        .join(artist[['Name', 'IsDutch', 'AgePassing', 'JaarTop2000', 'Overleden', 'EindeStemperiode']])
                                        .assign(DaysToStemperiode = lambda df: df['Overleden'].sub(df['EindeStemperiode']).dt.days,
                                                YearsSinceLastHit = lambda df: df['JaarTop2000'].sub(df['YearMade']),
                                                Boost = lambda df: df['PctVotes'].div(df['PctVotesBeforeDeath'])
                                               )

                    )

In [ ]:
combined = (analysis_overlijden.merge(artist_overlijden, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
            .assign(SongRelativeBoost = lambda df: df['BoostSong'].div(df['BoostArtist']),
                   LogRelativeBoost = lambda df: np.log2(df['SongRelativeBoost']))
           )

pd.concat([combined[['SongRelativeBoost', 'BoostSong']].quantile([0.025, 0.16, 0.5, 0.84, 0.975]).div(combined[['SongRelativeBoost', 'BoostSong']].median()),
           artist_overlijden['Boost'].quantile([0.025, 0.16, 0.5, 0.84, 0.975]).div(artist_overlijden['Boost'].median()).rename('BoostArtist')], axis='columns')

In [ ]:
sns.catplot(x='NrsBeforeDeath', y='LogRelativeBoost', data=combined)

## Univariate analysis
* Newsworthyness of artist passing
  * Age when passing away
  * Dutch nationality
  * Popularity
  * Year
  * Recency of hits
* Days to stemperiode

* Song
  * Popularity within artist
  * Recency of song within artist
  
### Newsworthyness

#### Artist age

In [ ]:
def plot_with_trend(df, column):
    preds = LinearRegression().fit(df[[column]], df['Boost']).predict(df[[column]])
    ax = df.plot(x=column, y='Boost', kind='scatter')
    ax.plot(df[column], preds, 'k--')
    return ax

In [ ]:
ax = plot_with_trend(artist_overlijden, 'AgePassing')

#### Jaar

In [ ]:
ax = plot_with_trend(artist_overlijden, 'JaarTop2000')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

#### Nationality

In [ ]:
_ = sns.catplot(x='IsDutch', y='Boost', data=artist_overlijden)

#### Recency

In [ ]:
ax = plot_with_trend(artist_overlijden.query('DaysToStemperiode > -365'), 'DaysToStemperiode')

In [ ]:
ax = plot_with_trend(artist_overlijden.query('DaysToStemperiode > -365').assign(DeathRecency= lambda df: np.exp(df['DaysToStemperiode']/100)), 'DeathRecency')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

### Recency of last hit

In [ ]:
ax = plot_with_trend(artist_overlijden, 'YearsSinceLastHit')

In [ ]:
ax = artist_overlijden.plot(x='YearsSinceLastHit', y='Boost', kind='scatter')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

#### Popularity

In [ ]:
ax = plot_with_trend(artist_overlijden.assign(LogPopularity = lambda df: np.log10(df['PctVotesBeforeDeath'])), 'LogPopularity')

### Song

### Popularity within artist oeuvre

In [ ]:
ax = plot_with_trend(analysis_overlijden.query('NrsBeforeDeath > 2'), 'PopularityWithinArtist')

### Recency within artist

In [ ]:
ax = plot_with_trend(analysis_overlijden.query('NrsBeforeDeath > 2'), 'RecencyWithinArtist')

In [ ]:
ax = plot_with_trend(analysis_overlijden, 'Recency')

## Multivariate

In [ ]:
song_boost.agg(['mean', 'sem', 'count'], axis='columns').plot(y='mean', yerr='sem')

In [ ]:
songartist.groupby('SongID').size()

In [ ]:
def find_before_death_song_popularity(songID):
    try:
        return notering.loc[(notering['SongID'] == songID) & (notering['Year'] == overlijdens_datum_per_song.loc[songID] - 1), 'PctVotes'].values[0]
    except IndexError:
        return 0

In [ ]:
song_effect_overlijden = (pd.concat([song_boost.loc[0].rename('OverlijdensBoost'),
                                     overlijdens_datum_per_song,
                                    song['YearMade']], axis='columns', join='inner')
                             .assign(BeforeDeathPopularity = lambda df: [find_before_death_song_popularity(songid) for songid in df.index],
                                    )
                            )

In [ ]:
song_effect_overlijden

In [ ]:
song_effect_overlijden.groupby(song_effect_overlijden['YearMade'] // 10 * 10)['OverlijdensBoost'].agg(['mean', 'sem', 'std', 'count']).plot(y='mean', yerr='sem')

In [ ]:
song_effect_overlijden.groupby('AantalArtiesten')['OverlijdensBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
song_effect_overlijden.groupby('JaarTop2000')['OverlijdensBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
song_effect_overlijden.plot(x='BeforeDeathPopularity', y='OverlijdensBoost', kind='scatter')

In [ ]:
song_effect_overlijden.groupby('AantalArtiesten')['OverlijdensBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
effect_overlijden = (pd.concat([song_boost.loc[0].rename('OverlijdensBoost'),
                                artist.loc[artist['IsOverleden'] == True, ['Name', 'Overleden', 'IsNederlands', 'EindeStemperiode', 'JaarTop2000']]],
                               axis='columns')
                     .assign(DaysToStemperiode = lambda df: df['Overleden'].sub(df['EindeStemperiode']).dt.days,
                            BeforeDeathPopularity = lambda df: [find_popularity_before_passing_away(artistname) for artistname in df.index],
                            LogPopularity = lambda df: np.log10(df['BeforeDeathPopularity']))
                     .replace([np.inf, -np.inf], np.nan)
                     .dropna()
                    )

In [ ]:
votes_per_year = analysis_solo_songs.groupby(['Year', 'ArtistID'])['PctVotes'].sum().unstack(fill_value=0)

In [ ]:
votes_per_year[votes_per_year.sum().nlargest(10).index].rename(columns=artist['Name'].to_dict()).plot(figsize=(20, 6))

In [ ]:
def mape_by_window(df, window=3):
    average = np.around(votes_per_year.rolling(window).mean().shift(), 5)
    mape = (votes_per_year.sub(average).abs()#.div(average) # 
             .melt()
             .replace([np.inf, -np.inf], np.nan).dropna()
             .mean()
           )
    return mape

In [ ]:
{i: mape_by_window(votes_per_year, i) for i in range(1, 5)}

In [ ]:
artist.rename(columns = lambda s: s.lower()).filter(regex='(?<!achtergrond)(?<!koor)zang')#.columns.tolist()

In [ ]:
artist[artist['gitarist, tweede leadzang'].notnull()]

In [ ]:
window=2
average = np.around(votes_per_year.rolling(window).mean().shift(), 5)
votes_per_year.sub(average).abs().div(average).melt().replace([np.inf, -np.inf], np.nan).dropna().mean()#ignore_index=False)

In [ ]:
# outc = votes_per_year.sub(average).abs().div(average).reset_index().melt(id_vars=['Jaar']).replace([np.inf, -np.inf], np.nan).dropna()

In [ ]:
# def normalize(series, years_to_normalize=3):
#     series.index -= series.idxmax()
#     series /= series.loc[-years_to_normalize:-1].mean()
#     return series

# normalize(votes_per_year['Ramses Shaffy'])

In [ ]:
artist['IsOverleden'].sum()

In [ ]:
artist

In [ ]:
overleden_artiest = artist[artist['IsOverleden'] == True].index

In [ ]:
overleden_artiest

In [ ]:
def normalize_by_year(series, years_to_normalize=3):
    series.index -= artist.loc[series.name, 'JaarTop2000']
    series /= series.loc[-years_to_normalize:-1].mean()
    return series

overleden_artiest = votes_per_year.columns.intersection(overleden_artiest)

votes_per_year_normalized = votes_per_year[overleden_artiest].apply(normalize_by_year, years_to_normalize=2)

In [ ]:
votes_per_year_normalized.replace(np.inf, np.nan).agg(['mean', 'sem', 'std', 'count'], axis='columns')#.plot(y='mean', yerr='std')

In [ ]:
def find_popularity_before_passing_away(artistname, nr_years=2):
    year_passing_away = artist.loc[artistname, 'JaarTop2000']
    popularity = votes_per_year.loc[year_passing_away-nr_years:year_passing_away-1, artistname].mean()
    return popularity

In [ ]:
effect_overlijden = (pd.concat([votes_per_year_normalized.loc[0].rename('OverlijdensBoost'),
                                artist.loc[artist['IsOverleden'] == True, ['Name', 'Overleden', 'IsNederlands', 'EindeStemperiode', 'JaarTop2000']]],
                               axis='columns', join='inner')
                     .assign(DaysToStemperiode = lambda df: df['Overleden'].sub(df['EindeStemperiode']).dt.days,
                            BeforeDeathPopularity = lambda df: [find_popularity_before_passing_away(artistname) for artistname in df.index],
                            LogPopularity = lambda df: np.log10(df['BeforeDeathPopularity']))
                     .sort_values('BeforeDeathPopularity')
                     .drop_duplicates('Overleden') # TODO: fix the double artists in a neat way

                     .replace([np.inf, -np.inf], np.nan)
                     .dropna()
                    )

In [ ]:
effect_overlijden

In [ ]:
def death_effect(x, minimum_effect=1.5, maximum_effect=3, days_start=30):
    return minimum_effect + x.add(days_start).clip(lower=0).div(days_start) * (maximum_effect - minimum_effect)

In [ ]:
popt, pcov = scipy.optimize.curve_fit(death_effect, effect_overlijden['DaysToStemperiode'], effect_overlijden['OverlijdensBoost'], [1.5, 3, 30])
perr = np.sqrt(np.diag(pcov))
popt

In [ ]:
perr

In [ ]:
recency_days = 50
effect_overlijden = effect_overlijden.assign(RecencyFactor = lambda df: df['DaysToStemperiode'].add(recency_days).clip(lower=0).div(recency_days)).sort_values('RecencyFactor')

In [ ]:
effect_overlijden.groupby('IsNederlands')['OverlijdensBoost'].agg(['mean', 'sem', 'count'])

In [ ]:
effect_overlijden.sort_values('OverlijdensBoost')

In [ ]:
effect_overlijden.plot(x='BeforeDeathPopularity', y='OverlijdensBoost', kind='scatter', logx=True)# y.assign().plot(x='LogPopularity'

In [ ]:
effect_overlijden.plot(x='JaarTop2000', y='OverlijdensBoost', kind='scatter', ylim=(0, 5))

In [ ]:
effect_overlijden.groupby('JaarTop2000')['OverlijdensBoost'].agg(['mean', 'sem'])#.plot(y='mean', yerr='sem')

In [ ]:
import statsmodels.api as sm

In [ ]:
effect_overlijden[['RecencyFactor', 'IsNederlands', 'JaarTop2000', 'LogPopularity']].corr()

In [ ]:
effect_overlijden.sort_values('OverlijdensBoost', ascending=False)

In [ ]:
y = effect_overlijden['OverlijdensBoost']
X = sm.add_constant(effect_overlijden.assign(IsNederlands = lambda df: df['IsNederlands'].astype(int),
                                             JaarTop2000 = lambda df: df['JaarTop2000'].sub(df['JaarTop2000'].max()),
                                            )
                    [['RecencyFactor', 'IsNederlands', 'JaarTop2000', 'LogPopularity']]
#                     [['RecencyFactor', 'JaarTop2000']]

                   )
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()

In [ ]:
effect_overlijden['PredictedBoost'] = res.predict(X)

In [ ]:
(effect_overlijden
 .assign(AbsoluteError = lambda df: df['OverlijdensBoost'].sub(df['PredictedBoost']).abs())
 .sort_values('AbsoluteError', ascending=False)
 .tail(20)
)#.plot(kind='scatter', x='DaysToStemperiode', y='AbsoluteError')

In [ ]:
effect_overlijden.plot(x='PredictedBoost', y='OverlijdensBoost', kind='scatter')
import matplotlib.pyplot as plt
plt.plot([0, 10], [0, 10], 'k--')

In [ ]:
effect_overlijden.groupby(effect_overlijden['Overleden'].dt.year)['OverlijdensBoost'].agg(['mean', 'sem', 'count'])#.rolling(3)['mean'].mean().plot()

In [ ]:
import datetime
import matplotlib.pyplot as plt
(effect_overlijden
 .plot(x='DaysToStemperiode', y='OverlijdensBoost', kind='scatter')
)
death_effect(pd.Series(range(-365, 1), index=range(-365, 1)), *popt).plot()
plt.axhline(1, c='k', ls='--')